# Predicting Spotify streams

In this assignment, you will analyze a dataset of the most popular songs on Spotify and try to predict the number of streams. You can find the dataset and a description of the variables here: https://www.kaggle.com/datasets/nelgiriyewithana/top-spotify-songs-2023

The dataset is also available to download from Brightspace. If you download it from Kaggle, note that the value in `df.loc[576, 'streams']` (cell I576 if you open it in Excel) is not a number so you need to deal with that in some way or you'll get errors when fitting the model. I've replaced it with a random number in the version on Brightspace.

Through the Kaggle link above, you can also find Python notebooks by other people that have analyzed the dataset! Feel free to go through them to get inspired, but remember that you should only hand in your own work. If you use large parts of code from an existing notebook, you have to put a comment above the code with a link to where you found it.

## The data

* The datafile `spotify-2023.csv` has 24 columns and 953 rows;
* For the variable descriptions, check the Kaggle link;
* The variable that you need to predict is `streams`. Note that we are predicting the number of streams, so this is a regression task!

## Compulsory elements of the task

1. [2 points] Load the dataset into a `pandas` dataframe, have a look at the first few lines. Briefly describe each column and the target variable. Are they numerical or categorical? What are the measurement units?
2. [2 points] The goal of the assignment is to make a prediction model.  Explain which variables you want to use, as well as which prediction model(s), and why.
3. [2 points] Show basic descriptives (mean, proportion, distribution, range, etc) of the variables that you plan to use in your prediction model, either numerically, graphically or a combination of the two.
4. [1 point] Clean the data, making sure data types are correct for each column. Convert categorical variables to dummy variables, check for and deal with missing data, etc.
5. [1 point] Create at least one additional column to use in your model.
6. [2 points] Use `.groupby()` or `.pivot_table()` in combination with row filtering at least once to demonstrate something related to your analysis.
7. [1 point] Split your data into a train and test set in a sensible way, and explain this split.  
8. [1 point] Train your model and predict the target value for the test set.
9. [3 points] Evaluate your results using relevant metrics to check the quality of your predictions. Investigate feature importances and whether they seem sensible to you.
10. [3 points] Write a conclusion: do you think you succeeded in building a good prediction model? Where could you improve? If you tried something that didn't work, why do you think it didn't work? Did you get the results you expected?


**Sum: 18 points. Partial points can be obtained for each element. Your grade will be equal to `1 + points/2`.**

Use your creativity and any method you'd like to try to achieve the best possible score for your prediction model. You can even use deep learning if you feel comfortable with that! Bonus points can be awarded for creativity.


## Getting started

To help you get started with this regression analysis, I've done a simple linear regression model on this dataset and evaluated it. Linear regression is quite a simple model, so your machine learning model will likely perform much better! Note that I have not done all the steps required for the assignment, your analysis should be much more in-depth!

In [1]:
import pandas as pd
df = pd.read_csv('spotify-2023.csv', encoding='latin-1') # Note that without specifying the encoding, pandas won't read the csv!
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 953 entries, 0 to 952
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   track_name            953 non-null    object
 1   artist(s)_name        953 non-null    object
 2   artist_count          953 non-null    int64 
 3   released_year         953 non-null    int64 
 4   released_month        953 non-null    int64 
 5   released_day          953 non-null    int64 
 6   in_spotify_playlists  953 non-null    int64 
 7   in_spotify_charts     953 non-null    int64 
 8   streams               953 non-null    int64 
 9   in_apple_playlists    953 non-null    int64 
 10  in_apple_charts       953 non-null    int64 
 11  in_deezer_playlists   953 non-null    object
 12  in_deezer_charts      953 non-null    int64 
 13  in_shazam_charts      903 non-null    object
 14  bpm                   953 non-null    int64 
 15  key                   858 non-null    ob

Currently, we cannot see all the columns, so we adjust the display to display all columns

In [2]:
pd.set_option('display.max_columns', None)
df.head(10)

,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,in_apple_charts,in_deezer_playlists,in_deezer_charts,in_shazam_charts,bpm,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%
0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2,2023,7,14,553,147,141381703,43,263,45,10,826,125,B,Major,80,89,83,31,0,8,4
1,LALA,Myke Towers,1,2023,3,23,1474,48,133716286,48,126,58,14,382,92,C#,Major,71,61,74,7,0,10,4
2,vampire,Olivia Rodrigo,1,2023,6,30,1397,113,140003974,94,207,91,14,949,138,F,Major,51,32,53,17,0,31,6
3,Cruel Summer,Taylor Swift,1,2019,8,23,7858,100,800840817,116,207,125,12,548,170,A,Major,55,58,72,11,0,11,15
4,WHERE SHE GOES,Bad Bunny,1,2023,5,18,3133,50,303236322,84,133,87,15,425,144,A,Minor,65,23,80,14,63,11,6
5,Sprinter,"Dave, Central Cee",2,2023,6,1,2186,91,183706234,67,213,88,17,946,141,C#,Major,92,66,58,19,0,8,24
6,Ella Baila Sola,"Eslabon Armado, Peso Pluma",2,2023,3,16,3090,50,725980112,34,222,43,13,418,148,F,Minor,67,83,76,48,0,8,3
7,Columbia,Quevedo,1,2023,7,7,714,43,58149378,25,89,30,13,194,100,F,Major,67,26,71,37,0,11,4
8,fukumean,Gunna,1,2023,5,15,1096,83,95217315,60,210,48,11,953,130,C#,Minor,85,22,62,12,0,28,9
9,La Bebe - Remix,"Peso Pluma, Yng Lvcas",2,2023,3,17,2953,44,553634067,49,110,66,13,339,170,D,Minor,81,56,48,21,0,8,33


So we have 24 Columns in total:
Categorical data:
1. track_name
2. artist(s)_name
3. mode
4. key

Numerical Data and their units:
- artist_count: number
- released_year: number(year)
- released_month: month
- released_day: day
- in_spotify_playlists: number
- in_spotify_charts: number
- streams: number
- in_apple_playlists: number
- in_apple_charts: number
- in_deezer_playlists: number
- in_deezer_charts: number
- in_shazam_charts: number
- bpm: beats per minute
- danceability_%: percentage
- valence_%: percentage
- energy_%: percentage
- acousticness_%: percentage
- instrumentalness_%: percentage
- liveness_%: percentage
- speechiness_%: percentage

I would love to use all possible variables, further more i would especially use the artist name since more popular artist get higher streams and vice versa. further more, the month  of release is also important because it could affect the song's stream count. especially if it summer or winter. Moreover, all the numerical data are useful and can together play a major role in my model

In [3]:
from sklearn.preprocessing import OneHotEncoder

cat_data = df[["key", "mode"]]


In [4]:
pd.get_dummies(cat_data, dummy_na= True, drop_first=True)

,key_A#,key_B,key_C#,key_D,key_D#,key_E,key_F,key_F#,key_G,key_G#,key_nan,mode_Minor,mode_nan
0,False,True,False,False,False,False,False,False,False,False,False,False,False
1,False,False,True,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,True,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
948,False,False,False,False,False,False,False,False,False,False,False,False,False
949,False,False,False,False,False,False,False,True,False,False,False,False,False
950,False,False,True,False,False,False,False,False,False,False,False,False,False
951,False,False,True,False,False,False,False,False,False,False,False,False,False


In [5]:
df_2 = cat_data


In [6]:
df_2

,key,mode
0,B,Major
1,C#,Major
2,F,Major
3,A,Major
4,A,Minor
...,...,...
948,A,Major
949,F#,Major
950,C#,Major
951,C#,Major


In [7]:
ohe = OneHotEncoder(categories = 'auto', drop = "first")

In [8]:
ohe.fit(df_2.fillna("Missing"))

OneHotEncoder(drop='first')

In [9]:
ohe.get_feature_names_out(["key","mode"])

array(['key_A#', 'key_B', 'key_C#', 'key_D', 'key_D#', 'key_E', 'key_F',
       'key_F#', 'key_G', 'key_G#', 'key_Missing', 'mode_Minor'],
      dtype=object)

In [10]:
df_3= ohe.transform(df_2.fillna("Missing")).toarray()

In [11]:
key_df = pd.DataFrame(df_3, columns=ohe.get_feature_names_out(["key","mode"])).astype(int)

In [12]:
df_4 = pd.concat([df, key_df], axis = 1)

In [13]:
df_5 = df_4.drop(columns=['key', 'mode'])
df_5

,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,in_apple_charts,in_deezer_playlists,in_deezer_charts,in_shazam_charts,bpm,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%,key_A#,key_B,key_C#,key_D,key_D#,key_E,key_F,key_F#,key_G,key_G#,key_Missing,mode_Minor
0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2,2023,7,14,553,147,141381703,43,263,45,10,826,125,80,89,83,31,0,8,4,0,1,0,0,0,0,0,0,0,0,0,0
1,LALA,Myke Towers,1,2023,3,23,1474,48,133716286,48,126,58,14,382,92,71,61,74,7,0,10,4,0,0,1,0,0,0,0,0,0,0,0,0
2,vampire,Olivia Rodrigo,1,2023,6,30,1397,113,140003974,94,207,91,14,949,138,51,32,53,17,0,31,6,0,0,0,0,0,0,1,0,0,0,0,0
3,Cruel Summer,Taylor Swift,1,2019,8,23,7858,100,800840817,116,207,125,12,548,170,55,58,72,11,0,11,15,0,0,0,0,0,0,0,0,0,0,0,0
4,WHERE SHE GOES,Bad Bunny,1,2023,5,18,3133,50,303236322,84,133,87,15,425,144,65,23,80,14,63,11,6,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
948,My Mind & Me,Selena Gomez,1,2022,11,3,953,0,91473363,61,13,37,1,0,144,60,24,39,57,0,8,3,0,0,0,0,0,0,0,0,0,0,0,0
949,Bigger Than The Whole Sky,Taylor Swift,1,2022,10,21,1180,0,121871870,4,0,8,0,0,166,42,7,24,83,1,12,6,0,0,0,0,0,0,0,1,0,0,0,0
950,A Veces (feat. Feid),"Feid, Paulo Londra",2,2022,11,3,573,0,73513683,2,0,7,0,0,92,80,81,67,4,0,8,6,0,0,1,0,0,0,0,0,0,0,0,0
951,En La De Ella,"Feid, Sech, Jhayco",3,2022,10,20,1320,0,133895612,29,26,17,0,0,97,82,67,77,8,0,12,5,0,0,1,0,0,0,0,0,0,0,0,0


In [14]:
df_5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 953 entries, 0 to 952
Data columns (total 34 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   track_name            953 non-null    object
 1   artist(s)_name        953 non-null    object
 2   artist_count          953 non-null    int64 
 3   released_year         953 non-null    int64 
 4   released_month        953 non-null    int64 
 5   released_day          953 non-null    int64 
 6   in_spotify_playlists  953 non-null    int64 
 7   in_spotify_charts     953 non-null    int64 
 8   streams               953 non-null    int64 
 9   in_apple_playlists    953 non-null    int64 
 10  in_apple_charts       953 non-null    int64 
 11  in_deezer_playlists   953 non-null    object
 12  in_deezer_charts      953 non-null    int64 
 13  in_shazam_charts      903 non-null    object
 14  bpm                   953 non-null    int64 
 15  danceability_%        953 non-null    in

In [15]:
# Assuming df_5 is your DataFrame and "in_shazam_charts" is the column containing float values

# Remove commas from the values in the column
df_5["in_shazam_charts"] = df_5["in_shazam_charts"].str.replace(',', '')

# Convert the column to float type
df_5["in_shazam_charts"] = df_5["in_shazam_charts"].astype(float)

# Calculate the mean of the column
mean_value = df_5["in_shazam_charts"].mean()

# Fill NaN values with the mean of the column
df_5["in_shazam_charts"].fillna(mean_value, inplace=True)

# Convert the column to int type
df_5["in_shazam_charts"] = df_5["in_shazam_charts"].astype(int)

df_5["in_deezer_playlists"] = df_5["in_deezer_playlists"].str.replace(',', '')

df_5["in_deezer_playlists"] = df_5["in_deezer_playlists"].astype(int)



C:\Users\moham\AppData\Local\Temp\ipykernel_20224\157252066.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_5["in_shazam_charts"].fillna(mean_value, inplace=True)


In [16]:
df_5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 953 entries, 0 to 952
Data columns (total 34 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   track_name            953 non-null    object
 1   artist(s)_name        953 non-null    object
 2   artist_count          953 non-null    int64 
 3   released_year         953 non-null    int64 
 4   released_month        953 non-null    int64 
 5   released_day          953 non-null    int64 
 6   in_spotify_playlists  953 non-null    int64 
 7   in_spotify_charts     953 non-null    int64 
 8   streams               953 non-null    int64 
 9   in_apple_playlists    953 non-null    int64 
 10  in_apple_charts       953 non-null    int64 
 11  in_deezer_playlists   953 non-null    int32 
 12  in_deezer_charts      953 non-null    int64 
 13  in_shazam_charts      953 non-null    int32 
 14  bpm                   953 non-null    int64 
 15  danceability_%        953 non-null    in

In [17]:
import pandas as pd

# Given array of artist names
artist_names = df_5["artist(s)_name"]

# Extract unique artist names
unique_artists = set()
for name in artist_names:
    artists = [artist.strip() for artist in name.split(',') if artist.strip()]
    unique_artists.update(artists)

unique_artists = sorted(list(unique_artists))

# Create DataFrame with columns for each unique artist
df_6 = pd.DataFrame(0, index=range(len(artist_names)), columns=unique_artists)

# Fill the DataFrame
for i, name in enumerate(artist_names):
    artists = [artist.strip() for artist in name.split(',') if artist.strip()]
    for artist in artists:
        df_6.at[i, artist] = 1

# Display the DataFrame
df_6


,(G)I-DLE,070 Shake,21 Savage,24kgoldn,2Pac,347aidan,4*TOWN (From Disney and Pixars Turning Red),50 Cent,A Boogie Wit da Hoodie,A$AP Rocky,ACRAZE,AP Dhillon,ARIZONATEARS,AX'EL,Abhijay Sharma,Abhir Hathi,Adassa,Adele,Aerosmith,AgroPlay,Agust D,Aitana,Aitch,Ak4:20,Alec Benjamin,Alejo,Alesso,Ali Sethi,Alicia Keys,Altamash Faridi,Alvaro Diaz,Amaarae,Amanda Reifer,Amitabh Bhattacharya,Amitabha Bhattacharya,Ana Castela,Anderson .Paak,Andy Williams,Angela Aguilar,Anggi Marito,Anitta,Anne-Marie,AnnenMayKantereit,Anuel Aa,Aqua,Arcane,Arcangel,Arctic Monkeys,Ariana Grande,Arijit Singh,Armani White,Ava,Aventura,Avicii,Ayparia,B,B. Swanson Quartet,BIGBANG,BL,BLACKPINK,BLESSD,BTS,BYOR,Baby Keem,Baby Rasta,Baby Tate,Bad B,Bad Bunny,Band Aid,Be,Beach House,Beach Weather,Beam,Bebe Rexha,Becky G,Becky Hill,Bellakath,Benny Blanco,Benson Boone,Beth Gibbons,Beyonc,Bianca,Big One,Billie Eilish,Bing Crosby,Bizarrap,Bjo,Black Eyed Peas,Blackbear,Blake Shelton,Blanco,Blxst,Bobby Helms,Bomba Estreo,BoyWithUke,Bradley Cooper,Brenda Lee,Britney Spears,Brray,Bruno Mars,Burl Ives,Burna Boy,Buscabulla,Bvga Beatz,C. Tangana,Calum Scott,Calvin Harris,Camila Cabello,Caralisa Monteiro,Cardi B,Carin Leon,Carolina Gaitan,Cartel De Santa,Central Cee,Chanel,Charli XCX,Charlie Puth,Chase Atlantic,Chefin,Chencho Corleone,Cherish,Childish Gambino,Chino Pacas,Chris Brown,Chris Jedi,Chris Molitor,Chris Rea,Christian Nodal,Chuck Berry,Cigarettes After Sex,Ckay,Coi Leray,Colde,Coldplay,Colin Hay,Conan Gray,Coolio,Creedence Clearwater Revival,Cris Mj,Cruz Cafun,DJ 900,DJ Biel do Furduncinho,DJ Escobar,DJ Khaled,DJ Luian,DJ Matt D,DJ R,DJ Win,DaBaby,Daddy Yankee,Daft Punk,Dalex,Daniel Caesar,Danny Ocean,Darlene Love,Dave,Davi K,David Guetta,David Kushner,Dazy,De La Ghetto,Dean Lewis,Dean Martin,Demi Lovato,Diane Guerrero,Dido,Dimelo Flow,Dina Rae,Dj Chris No Beat,Dj Gabriel do Borel,Dj LK da Esccia,Dj Yo!,Doechii,Doja Cat,Don Omar,Don Toliver,Dopamine,Dou,Dove Cameron,Dr. Dre,Drake,Dua Lipa,Duki,Duncan Laurence,ENHYPEN,Ed Sheeran,Eden Mu,Edison Lighthouse,El Chachito,Eladio Carrion,Eliza Rose,Ella Henderson,Elley Duh,Ellie Goulding,Elton John,Em Beihold,Emilia,Eminem,Emmy Meli,Encanto - Cast,Eric Church,Eslabon Armado,FIFA Sound,FMK,Farruko,Feid,Fifty Fifty,Finneas O'Connell,Fireboy DML,Fivio Foreign,Flowdan,Foudeqush,Fran C,Frank Ocean,Frank Sinatra,Fred again..,Freddie Dredd,Fuerza Regida,Fujii Kaze,Future,GODZZ__-,Gabito Ballesteros,Gabry Ponte,Gayle,George Henrique &,Gesaffelstein,Ghost,Ghostface Killah,Giant Rooks,Giveon,Glass Animals,Gorillaz,Gotye,Grayson Vill,Grupo Frontera,Grupo Marca Registrada,Gunna,Guns N' Roses,Gurinder Gill,Gustah,Gustavo Mioto,Gwen Stefani,HA SUNG WOON,Hallie Coggins,Halsey,Harry Styles,Hotel Ugly,Hozier,Hugo & G,Huilen Toledo,INTERWORLD,IU,IVE,Iann Dior,Ice Spice,Imagine Dragons,Imanbek,Intense,Interplanetary Criminal,Israel & Rodolffo,J Balvin,J. Cole,JID,JISOO,JVKE,Jack Black,Jack Harlow,Jain,James Arthur,James Blake,James Hype,Jasiel Nuez,Jason Derulo,Jax Jones,Jay Rock,Jaymes Young,Jessi,Jessica Darrow,Jessie Reyez,Jhay Cortez,Jhayco,Jimin,Jin,Jnr Choi,Joel Corry,John Legend,John Lennon,John Scott Trotter & His Orchestra,Joji,Jonas Brothers,Jordan Fisher,Jos Felic,Josh Levi,Juan Cruz Toledo,Juice WRLD,Juicy J,Julieta Venegas,Jung Kook,Junior H,Juseph,Justin Bieber,Justin Quiles,Justin Timberlake,KALUSH,Kaifi Khalil,Kali Uchis,Kaliii,Kanii,Kanye West,Kar,Karol G,Kate Bush,Katy Perry,KayBlack,Ke personajes,Keane,Keisya Levronka,Kelly Clarkson,Ken Darby Singers,Kendrick Lamar,Kenia OS,Kenshi Yonezu,Kevin Gates,Kevin Kaarl,Khalid,Kim Petras,Kimbra,King,Kodak Black,Kordhell,Kumaar,Kyla,L-Gante,L.V.,L7nnon,LE SSERAFIM,LF System,LUM!X,La Adictiva,La Joaqui,La Kelly,La Pantera,Labrinth,Lady Gaga,Lana Del Rey,Lasso,Latto,Lauren Spencer Smith,League of Legends,Leah Kate,Lenny Tavrez,Leo Santana,Lewis Capaldi,LiL CaKe,Libianca,Lil Baby,Lil Durk,Lil Nas X,Lil Tjay,Lil Uzi Vert,Lil Wayne,Lil Yachty,Linkin Park,Lisa,Lit Killah,Lizzo,Lizzy McAlpine,Lo

In [18]:
df_7 = pd.concat([df_5, df_6], axis = 1)
df_7.drop(columns=["artist(s)_name"])

,track_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,in_apple_charts,in_deezer_playlists,in_deezer_charts,in_shazam_charts,bpm,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%,key_A#,key_B,key_C#,key_D,key_D#,key_E,key_F,key_F#,key_G,key_G#,key_Missing,mode_Minor,(G)I-DLE,070 Shake,21 Savage,24kgoldn,2Pac,347aidan,4*TOWN (From Disney and Pixars Turning Red),50 Cent,A Boogie Wit da Hoodie,A$AP Rocky,ACRAZE,AP Dhillon,ARIZONATEARS,AX'EL,Abhijay Sharma,Abhir Hathi,Adassa,Adele,Aerosmith,AgroPlay,Agust D,Aitana,Aitch,Ak4:20,Alec Benjamin,Alejo,Alesso,Ali Sethi,Alicia Keys,Altamash Faridi,Alvaro Diaz,Amaarae,Amanda Reifer,Amitabh Bhattacharya,Amitabha Bhattacharya,Ana Castela,Anderson .Paak,Andy Williams,Angela Aguilar,Anggi Marito,Anitta,Anne-Marie,AnnenMayKantereit,Anuel Aa,Aqua,Arcane,Arcangel,Arctic Monkeys,Ariana Grande,Arijit Singh,Armani White,Ava,Aventura,Avicii,Ayparia,B,B. Swanson Quartet,BIGBANG,BL,BLACKPINK,BLESSD,BTS,BYOR,Baby Keem,Baby Rasta,Baby Tate,Bad B,Bad Bunny,Band Aid,Be,Beach House,Beach Weather,Beam,Bebe Rexha,Becky G,Becky Hill,Bellakath,Benny Blanco,Benson Boone,Beth Gibbons,Beyonc,Bianca,Big One,Billie Eilish,Bing Crosby,Bizarrap,Bjo,Black Eyed Peas,Blackbear,Blake Shelton,Blanco,Blxst,Bobby Helms,Bomba Estreo,BoyWithUke,Bradley Cooper,Brenda Lee,Britney Spears,Brray,Bruno Mars,Burl Ives,Burna Boy,Buscabulla,Bvga Beatz,C. Tangana,Calum Scott,Calvin Harris,Camila Cabello,Caralisa Monteiro,Cardi B,Carin Leon,Carolina Gaitan,Cartel De Santa,Central Cee,Chanel,Charli XCX,Charlie Puth,Chase Atlantic,Chefin,Chencho Corleone,Cherish,Childish Gambino,Chino Pacas,Chris Brown,Chris Jedi,Chris Molitor,Chris Rea,Christian Nodal,Chuck Berry,Cigarettes After Sex,Ckay,Coi Leray,Colde,Coldplay,Colin Hay,Conan Gray,Coolio,Creedence Clearwater Revival,Cris Mj,Cruz Cafun,DJ 900,DJ Biel do Furduncinho,DJ Escobar,DJ Khaled,DJ Luian,DJ Matt D,DJ R,DJ Win,DaBaby,Daddy Yankee,Daft Punk,Dalex,Daniel Caesar,Danny Ocean,Darlene Love,Dave,Davi K,David Guetta,David Kushner,Dazy,De La Ghetto,Dean Lewis,Dean Martin,Demi Lovato,Diane Guerrero,Dido,Dimelo Flow,Dina Rae,Dj Chris No Beat,Dj Gabriel do Borel,Dj LK da Esccia,Dj Yo!,Doechii,Doja Cat,Don Omar,Don Toliver,Dopamine,Dou,Dove Cameron,Dr. Dre,Drake,Dua Lipa,Duki,Duncan Laurence,ENHYPEN,Ed Sheeran,Eden Mu,Edison Lighthouse,El Chachito,Eladio Carrion,Eliza Rose,Ella Henderson,Elley Duh,Ellie Goulding,Elton John,Em Beihold,Emilia,Eminem,Emmy Meli,Encanto - Cast,Eric Church,Eslabon Armado,FIFA Sound,FMK,Farruko,Feid,Fifty Fifty,Finneas O'Connell,Fireboy DML,Fivio Foreign,Flowdan,Foudeqush,Fran C,Frank Ocean,Frank Sinatra,Fred again..,Freddie Dredd,Fuerza Regida,Fujii Kaze,Future,GODZZ__-,Gabito Ballesteros,Gabry Ponte,Gayle,George Henrique &,Gesaffelstein,Ghost,Ghostface Killah,Giant Rooks,Giveon,Glass Animals,Gorillaz,Gotye,Grayson Vill,Grupo Frontera,Grupo Marca Registrada,Gunna,Guns N' Roses,Gurinder Gill,Gustah,Gustavo Mioto,Gwen Stefani,HA SUNG WOON,Hallie Coggins,Halsey,Harry Styles,Hotel Ugly,Hozier,Hugo & G,Huilen Toledo,INTERWORLD,IU,IVE,Iann Dior,Ice Spice,Imagine Dragons,Imanbek,Intense,Interplanetary Criminal,Israel & Rodolffo,J Balvin,J. Cole,JID,JISOO,JVKE,Jack Black,Jack Harlow,Jain,James Arthur,James Blake,James Hype,Jasiel Nuez,Jason Derulo,Jax Jones,Jay Rock,Jaymes Young,Jessi,Jessica Darrow,Jessie Reyez,Jhay Cortez,Jhayco,Jimin,Jin,Jnr Choi,Joel Corry,John Legend,John Lennon,John Scott Trotter & His Orchestra,Joji,Jonas Brothers,Jordan Fisher,Jos Felic,Josh Levi,Juan Cruz Toledo,Juice WRLD,Juicy J,Julieta Venegas,Jung Kook,Junior H,Juseph,Justin Bieber,Justin Quiles,Justin Timberlake,KALUSH,Kaifi Khalil,Kali Uchis,Kaliii,Kanii,Kanye West,Kar,Karol G,Kate Bush,Katy Perry,KayBlack,Ke personajes,Keane,Keisya Levronka,Kelly Clarkson,Ken Darby Singers,Kendrick Lamar,Kenia OS,Kenshi Yonezu,Kevin Gates,Kevin Kaarl,Khalid,Kim Petras,Kimbra,King,Kodak Bl

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Step 1: Data Preprocessing
X = df_7.drop(["streams", "track_name", "artist(s)_name"], axis=1)
y = df_7["streams"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 2: Choose a Model
model = RandomForestRegressor(random_state=42)

# Step 3: Train the Model
model.fit(X_train, y_train)

# Step 4: Evaluate the Model
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)
print("R-squared:", r2)

# Step 5: Fine-tune the Model (if necessary)
# - Random forests have hyperparameters such as the number of trees, max depth of trees, etc., that you can tune using techniques like grid search or random search.

# Step 6: Make Predictions
# - Use the trained model to make predictions on new data.


Mean Absolute Error: 134440501.10643977
Mean Squared Error: 4.250206355053722e+16
R-squared: 0.8431241742766468


In [ ]:
from sklearn.tree import DecisionTreeRegressor

# Step 1: Data Preprocessing
X = df_7.drop(["streams", "track_name", "artist(s)_name"], axis=1)
y = df_7["streams"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 2: Choose a Model
model = DecisionTreeRegressor(random_state=42)

# Step 3: Train the Model
model.fit(X_train, y_train)

# Step 4: Evaluate the Model
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)
print("R-squared:", r2)

# Step 5: Fine-tune the Model (if necessary)
# - Decision trees also have hyperparameters like max depth, min samples split, etc., that you can adjust to improve performance.

# Step 6: Make Predictions
# - Use the trained model to make predictions on new data.


In [ ]:
# Create a new variable - we will create a variable that equals 1 when the song is by
# Taylor Swift, and 0 if it is not
def TSinartists(artists):
  if 'Taylor Swift' in artists:
    return 1
  else:
    return 0

df['TaylorSwift'] = df['artist(s)_name'].map(TSinartists)

# Keep only the variables we want to use: artist_count, released_month, danceability_%, and TaylorSwift
# and of course what we want to predict: streams!
df2 = df[['artist_count', 'released_month', 'danceability_%', 'TaylorSwift', 'streams']]
df2.head()

Please note that I am only selecting four predictor variables to show you this example. For your analysis, only exclude columns if you have good reason to! If it is relevant and could add information to the model, don't just leave it out.

In [ ]:
# Make sure all variables are numerical
df2.info()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import median_absolute_error

# Split in train and test
train, test = train_test_split(df2, test_size=0.3)
X_train = train[['artist_count', 'released_month', 'danceability_%', 'TaylorSwift']]
X_test = test[['artist_count', 'released_month', 'danceability_%', 'TaylorSwift']]
y_train = train['streams']
y_test = test['streams']

# Fit the model to the train data
model = LinearRegression()
model.fit(X_train, y_train)

# Predict streams for test data
y_predicted = model.predict(X_test)

# Calculate error (lower is better, see https://scikit-learn.org/stable/modules/generated/sklearn.metrics.median_absolute_error.html#sklearn.metrics.median_absolute_error)
print(f'MAE is {round(median_absolute_error(y_test, y_predicted))}')

We are using the median absolute error (MAE) to assess our model's prediction accuracy. Note that this value is not very interpretable on its own, unlike simple accuracy, when we are doing classification. Interpret the MAE as follows: for every observation in the test set, we calculate the absolute difference between the true and predicted value. Then we take the median value of all those absolute differences. This is equal to 317 million, so the predictions are off by a lot of streams!

Let's visualize the predictions. The straight orange line is the line that all points would lie on if all predicted values were exactly correct. We see that this model is not good at predicting streams at all! The reason is that linear regression models only work well if there is a linear relationship between the number of streams and the variables we use to predict them. Apparently this is not the case, so we need a better model! I will leave that up to you 😺



In [ ]:
import matplotlib.pyplot as plt

plt.scatter(y_test, y_predicted)
plt.xlabel('Actual number of streams')
plt.ylabel('Predicted number of streams')
plt.plot([0,1000000000],[0,1000000000], c='orange')
plt.show()